# Test isochrone output

I have a json of outputs returned by Azure Maps. Let's see if it is usable before I spend a bunch of cycles querying a bunch of results

In [ ]:
from pathlib import Path
import json
import folium
import geopandas as gpd
from shapely.geometry import shape, Point, Polygon

In [ ]:
data_dir = Path("../data/isochrone")

In [ ]:
data_dir.exists()

In [ ]:
# Turns out azure isochrones kind of suck
# with open(data_dir / "azure_isochrones.json", "r") as f:
#     azure_isochrones = json.load(f)

In [ ]:
with open(data_dir / "isochrones.json", "r") as f:
    isochrones = json.load(f)

In [ ]:
center = [51.04487312693333, -114.07864049159537]

In [ ]:
for isochrone in isochrones:
    isochrone["shape"] = shape(isochrone["shape"])

In [ ]:
# for isochrone in azure_isochrones:
#     isochrone["shape"] = Polygon([Point(point["longitude"], point["latitude"]) for point in isochrone["shape"]])

In [ ]:
my_geo_df = gpd.GeoDataFrame(isochrones).set_index(["place_name", "mode", "cutoff_time"])

In [ ]:
# my_azure_df = gpd.GeoDataFrame(azure_isochrones).set_index(["place_name", "mode", "cutoff_time"])

In [ ]:
# ['GF_WORK', 'BROTHER', 'DOWNTOWN', 'MACEWAN', "CURRENT_HOUSE", "WORK"]
m = folium.Map(location=center)
PLACE_NAME = "GF_WORK"
# ['WALK, TRANSIT', 'WALK', 'CAR', 'BICYCLE', 'WALK, BICYCLE, TRANSIT']
MODE = "CAR"
# 10 to 60 in 5 minute intervals
TIME = 20
geo = gpd.GeoSeries(my_geo_df.loc[(PLACE_NAME, MODE, TIME), "shape"])
geo_j = geo.to_json()
geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': 'orange'})
folium.Popup(f"{PLACE_NAME}, {MODE}, {TIME}").add_to(geo_j)
geo_j.add_to(m)
# if MODE == "CAR":
#     geo2 = gpd.GeoSeries(my_azure_df.loc[(PLACE_NAME, "azure_car", TIME), "shape"])
#     geo_j2 = geo2.to_json()
#     geo_j2 = folium.GeoJson(data=geo_j2, style_function=lambda x: {'fillColor': 'blue'})
#     folium.Popup(f"{PLACE_NAME}, azure_car, {TIME}").add_to(geo_j2)
#     geo_j2.add_to(m)
m
